Plot the spectrum of the estimated graphs for different types of constraints.

In [1]:
import cvxpy as cp
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import time

import utils
import spectral_nti as snti

%matplotlib qt

SEED = 0
np.random.seed(SEED)

In [14]:
def est_graphs(models, C_hat):
    N = C_hat.shape[0]
    A_hat = np.zeros((N, N, len(models)))
    lamd_hat = np.zeros((N,len(models)))

    for j, model in enumerate(models):
        t = time.time()
        L_hat, _ = \
            snti.MGL(C_hat, model['gs'], model['bounds'],
                        model['cs'], model['regs'], max_iters=iters)
            
        A_hat[:,:,j] = np.diag(np.diag(L_hat)) - L_hat
        lamd_hat[:,j], _ = np.linalg.eigh(L_hat)
        t = time.time() - t
        print('Model: {} - Time(sec): {:.3}'.format(model['name'], t))
    return A_hat, lamd_hat


def print_err(models, A, As_hat):
    L = np.diag(np.sum(A,0)) - A
    norm_A = np.linalg.norm(A,'fro')
    norm_L = np.linalg.norm(L,'fro')

    for j, model in enumerate(models):
        L_hat = np.diag(np.sum(As_hat[:,:,j],0)) - As_hat[:,:,j]
        errA = np.linalg.norm(A-As_hat[:,:,j],'fro')**2/norm_A**2
        errL = np.linalg.norm(L-L_hat,'fro')**2/norm_L**2

        print('{}:\tErr A: {:.4f} - Err L: {:.4f}'
              .format(model['name'], errA, errL))


def print_err2(models, A, As_hat):
    L = np.diag(np.sum(A,0)) - A
    norm_A = np.linalg.norm(A, 'fro')
    norm_L = np.linalg.norm(L, 'fro')

    for j, model in enumerate(models):
        L_hat = np.diag(np.sum(As_hat[:,:,j],0)) - As_hat[:,:,j]
        L_hat /= np.linalg.norm(L_hat, 'fro')
        A_hat = As_hat[:,:,j]/np.linalg.norm(As_hat[:,:,j], 'fro')
        errA = np.linalg.norm(A/norm_A-A_hat,'fro')**2
        errL = np.linalg.norm(L/norm_L-L_hat,'fro')**2

        print('{}:\tErr2 A: {:.4f} - Err2 L: {:.4f}'
              .format(model['name'], errA, errL))


def plot_graphs(models, A, As_hat):
    plt.figure()
    plt.imshow(A)
    plt.title('Original A')
    plt.colorbar()

    for i,model in enumerate(models):
        plt.figure()
        plt.imshow(As_hat[:,:,i])
        plt.title(model['name'])
        plt.colorbar()
  

def plot_spectrum(models, lambdas, lambdas0, lamds_hat, M):
    n_bins = np.arange(0,np.amax(lamds_hat)+1,1)

    fig, axs = plt.subplots(2, 4, sharey=True, constrained_layout=True)
    axs[0,0].hist(lambdas, density=True, bins=n_bins)
    axs[0,0].set_title('True spectrum')
    axs[1,0].hist(lambdas0, density=True, bins=n_bins)
    axs[1,0].set_title('Ref spectrum')
    axs[0,1].hist(lamds_hat[:,0], density=True, bins=n_bins)
    axs[0,1].set_title(models[0]['name'])
    axs[0,2].hist(lamds_hat[:,3], density=True, bins=n_bins)
    axs[0,2].set_title(models[3]['name'])
    axs[0,3].hist(lamds_hat[:,4], density=True, bins=n_bins)
    axs[0,3].set_title(models[4]['name'])
    axs[1,1].hist(lamds_hat[:,1], density=True, bins=n_bins)
    axs[1,1].set_title(models[1]['name'])
    axs[1,2].hist(lamds_hat[:,2], density=True, bins=n_bins)
    axs[1,2].set_title(models[2]['name'])
    axs[1,3].hist(lamds_hat[:,5], density=True, bins=n_bins)
    axs[1,3].set_title(models[5]['name'])
    fig.suptitle('M=' + str(M))


## Create grid graphs

In [3]:
# Ref grap - 150 nodes
n01 = 15
n02 = 10
N0 = n01*n02
A0 = nx.to_numpy_array(nx.grid_2d_graph(n01, n02))
L0 = np.diag(np.sum(A0, 0)) - A0
lambdas0, _ = np.linalg.eigh(L0)

# Target graph - 200 nodes
n1 = 20
n2 = 10
N = n1*n2
A = nx.to_numpy_array(nx.grid_2d_graph(n1, n2))
L = np.diag(np.sum(A, 0)) - A
lambdas, V = np.linalg.eigh(L)
A_norm = np.linalg.norm(A,'fro')**2
lambdas_norm = np.linalg.norm(lambdas)**2

norm_A = np.linalg.norm(A,'fro')
norm_lamb = np.linalg.norm(lambdas)

Note: Although graph images are shown in different colors I have checked that they are both binary graphs. Must be an error of the library.

## Models

In [4]:
iters = 200

GS = [
    lambda a, b : cp.sum(a)/b,    # delta: 4e-2
    lambda a, b : cp.sum(a**2)/b,  # delta: .7
    lambda a, b : cp.sum(cp.exp(-a))/b,    # delta: 3e-3
    lambda a, b : cp.sum(cp.sqrt(a))/b,  # delta: 2e-2
    lambda a, b : cp.sum(.25*a**2-.75*a)/b,
]
BOUNDS = [
    lambda lamd, lamd_t, b : -2/b*lamd_t.T@lamd,
    lambda lamd, lamd_t, b : 1/b*cp.exp(-lamd_t).T@lamd,
    lambda lamd, lamd_t, b : cp.sum(lamd/cp.sqrt(lamd_t))/(2*b),
    lambda lamd, lamd_t, b: 1/b*(0.75-2*0.25*lamd_t).T@lamd,
]

deltas = [.04, .27, .003, .02, 0.05]
cs = utils.compute_cs(GS, lambdas0, lambdas)

models = [
    {'name': 'Unc', 'gs': [], 'bounds': [], 'cs': [], 'fmt': '1-',
     'regs': {'alpha': .01, 'beta': 1.75, 'gamma': 0, 'deltas': 0}},
    
    {'name': 'Tr', 'gs': GS[0], 'bounds': [], 'cs': cs[0], 'fmt': '2-',
     'regs': {'alpha': 0, 'beta': 1, 'gamma': 0, 'deltas': deltas[0]}},

    {'name': 'Sq', 'gs': GS[1], 'bounds': BOUNDS[0], 'cs': cs[1], 'fmt': 'o-',
     'regs': {'alpha': .0, 'beta': 1.25, 'gamma': 1000, 'deltas': deltas[1]}},

    {'name': 'Heat', 'gs': GS[2], 'bounds': BOUNDS[1], 'cs': cs[2], 'fmt': 'x-',
     'regs': {'alpha': .001, 'beta': .4, 'gamma': 100, 'deltas': deltas[2]}},
    
    {'name': 'Sqrt', 'gs': GS[3], 'bounds': BOUNDS[2], 'cs': cs[3], 'fmt': 'v-',
     'regs': {'alpha': .01, 'beta': .5, 'gamma': 25, 'deltas': deltas[3]}},

    {'name': 'Poly', 'gs': GS[4], 'bounds': BOUNDS[3], 'cs': cs[4], 'fmt': 's-',
     'regs': {'alpha': 0, 'beta': 1, 'gamma': 1000, 'deltas': deltas[4]}},
]

	c-0: c: 3.700	c0: 3.667	err: 0.033333	err norm: 0.009091
	c-1: c: 17.640	c0: 17.387	err: 0.253333	err norm: 0.014571
	c-2: c: 0.114	c0: 0.115	err: -0.001259	err norm: -0.010911
	c-3: c: 1.828	c0: 1.817	err: 0.010217	err norm: 0.005622
	c-4: c: 1.635	c0: 1.597	err: 0.038333	err norm: 0.024008


## Estimate graphs

In [5]:
MM = [500, 750, 1000]

lambdas_aux = np.concatenate(([0], 1/np.sqrt(lambdas[1:])))
C_inv_sqrt = V@np.diag(lambdas_aux)@V.T
X = C_inv_sqrt@np.random.randn(N, MM[-1])

As_hat = np.zeros((N, N, len(models), len(MM)))
lamds_hat = np.zeros((N, len(models), len(MM)))
for i,M in enumerate(MM):
    C_hat = X[:,:M]@X[:,:M].T/M

    # Estimate graph
    total_t = time.time()
    As_hat[:,:,:,i], lamds_hat[:,:,i] = est_graphs(models, C_hat)

Model: Unc - Time(sec): 14.6


C:\Users\Samuel\anaconda3\lib\site-packages\cvxpy\problems\problem.py:1267: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Model: Tr - Time(sec): 15.9
Model: Sq - Time(sec): 17.4
Model: Heat - Time(sec): 16.8
Model: Sqrt - Time(sec): 17.8
Model: Poly - Time(sec): 18.5
Model: Unc - Time(sec): 15.0
Model: Tr - Time(sec): 16.0
Model: Sq - Time(sec): 17.9
Model: Heat - Time(sec): 17.3
Model: Sqrt - Time(sec): 17.3
Model: Poly - Time(sec): 18.1
Model: Unc - Time(sec): 14.7
Model: Tr - Time(sec): 15.6
Model: Sq - Time(sec): 17.9
Model: Heat - Time(sec): 17.4
Model: Sqrt - Time(sec): 17.5
Model: Poly - Time(sec): 17.9


In [17]:
for i,M in enumerate(MM):
    print('M:', M)
    print_err2(models, A, As_hat[:,:,:,i])
    plot_spectrum(models, lambdas, lambdas0, lamds_hat[:,:,i], M)


M: 500
Unc:	Err2 A: 0.2413 - Err2 L: 0.1038
Tr:	Err2 A: 0.1441 - Err2 L: 0.0637
Sq:	Err2 A: 0.1134 - Err2 L: 0.0420
Heat:	Err2 A: 0.0934 - Err2 L: 0.0403
Sqrt:	Err2 A: 0.0973 - Err2 L: 0.0423
Poly:	Err2 A: 0.1083 - Err2 L: 0.0406
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23.]
M: 750
Unc:	Err2 A: 0.0822 - Err2 L: 0.0404
Tr:	Err2 A: 0.0566 - Err2 L: 0.0256
Sq:	Err2 A: 0.0468 - Err2 L: 0.0183
Heat:	Err2 A: 0.0453 - Err2 L: 0.0203
Sqrt:	Err2 A: 0.0469 - Err2 L: 0.0213
Poly:	Err2 A: 0.0453 - Err2 L: 0.0181
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
M: 1000
Unc:	Err2 A: 0.0444 - Err2 L: 0.0203
Tr:	Err2 A: 0.0344 - Err2 L: 0.0150
Sq:	Err2 A: 0.0290 - Err2 L: 0.0112
Heat:	Err2 A: 0.0309 - Err2 L: 0.0133
Sqrt:	Err2 A: 0.0316 - Err2 L: 0.0137
Poly:	Err2 A: 0.0284 - Err2 L: 0.0111
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
